In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Final_file_path = "preprocessed_data_option1.csv"
global data
data = pd.read_csv(Final_file_path)

C:\Users\simon\AppData\Local\Temp\ipykernel_18956\1351336297.py:10: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(Final_file_path)


In [2]:
# Print alle unikke værdier i 'type' kolonnen
print(data['type'].unique())

# Fjern rækker hvor 'type' kolonnen har bestemte værdier
global data_
data = data[~data['type'].isin(['satire', 'conspiracy', 'junksci', 'unreliable', 'rumor', 'unknown', 'nan', 'hate', "<NUM><DATE> <NUM>:<NUM>:<NUM>"])]
data = data.dropna(subset=["type"])

# Print alle unikke værdier i 'type' kolonnen efter fjernelse
print(data['type'].unique())

# Definér mapping
label_mapping = {
    "political": "True",
    "clickbait": "True",
    "reliable": "True",
    "fake": "Fake",
    "bias": "Fake"
}

# Opret label-kolonnen først
data["label"] = data["type"].map(label_mapping)

# Konverter 'True' til 1 og 'Fake' til 0
data["label"] = data["label"].map({"True": 1, "Fake": 0})


# Definér y (målvariabel)
global y
y = data["label"].dropna().astype(int)

# Tjek om det virker
print(y.unique())  # Skal vise [1 0]


['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'unknown' nan 'clickbait' 'hate' 'junksci'
 '<NUM><DATE> <NUM>:<NUM>:<NUM>']
['political' 'fake' 'reliable' 'bias' 'clickbait']
[1 0]


In [4]:
data["content"] = data["content"].fillna("").astype(str)

vectorizer = TfidfVectorizer(max_features=10000, binary=True)
X = vectorizer.fit_transform(data["content"])
# splitting
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# implementing the logistic regression
clf = LogisticRegression(max_iter=500, random_state=42, solver="saga", n_jobs=-1)
clf.fit(X_train, y_train)

# evaluate the model
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)

print(f"F1-score: {f1:.4f}")
print("Hyperparametre for Logistic Regression:")
print(f"Max Iterations: {clf.max_iter}")
print(f"Solver: {clf.solver}")
print(f"Random State: {clf.random_state}")


KeyboardInterrupt: 